In [ ]:
# Import required modules
import torch
import torch.nn.functional as F
from skimage import img_as_ubyte

from Restormer.basicsr.models.archs.restormer_arch import Restormer

In [ ]:
from PyQt5.QtWidgets import (QMainWindow, QGroupBox, QAction, QStyle, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, QTextEdit, QFileDialog, QApplication, QStatusBar,
                             QComboBox, QDesktopWidget, QMessageBox, QSizePolicy, QScrollArea, QSlider, QGridLayout, QCheckBox)
from PyQt5.QtGui import QPixmap, QImage, QPalette, QColor, QPainter, QFont
from PyQt5.QtCore import Qt, QDateTime, QThread, pyqtSignal
from PIL import Image
import pytesseract
import speech_recognition as sr
from pdf2image import convert_from_path
from docx import Document

import sys
import os
import pyaudio
import wave
import re
import datetime
import json

import cv2
import numpy as np

import os
# os.environ["QT_QPA_PLATFORM"] = "xcb"

class RecordingThread(QThread):
    """Thread to handle audio recording without blocking the GUI"""
    finished = pyqtSignal(str, str)  # Will emit transcription text and filepath
    
    def __init__(self, filename):
        super().__init__()
        self.filename = filename
        self.running = True
        
    def run(self):
        # Audio recording parameters
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 44100
        CHUNK = 1024
        
        audio = pyaudio.PyAudio()
        
        # Start recording
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)
        
        frames = []
        
        while self.running:
            data = stream.read(CHUNK)
            frames.append(data)
        
        # Stop recording
        stream.stop_stream()
        stream.close()
        audio.terminate()
        
        # Save the recording
        wf = wave.open(self.filename, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        # Transcribe
        recognizer = sr.Recognizer()
        with sr.AudioFile(self.filename) as source:
            audio_data = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_data)
                self.finished.emit(text, self.filename)
            except sr.UnknownValueError:
                self.finished.emit("Speech recognition could not understand audio", self.filename)
            except sr.RequestError as e:
                self.finished.emit(f"Could not request results; {e}", self.filename)
        
    def stop(self):
        self.running = False


class MultiFOLD(QMainWindow):  # Changed to QMainWindow for proper menu bar
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        ################################# WINDOW SETUP #################################
        # Get screen size (primary screen)
        screen_geometry = QDesktopWidget().screenGeometry()

        # Set window title and size relative to screen size
        self.setWindowTitle('MultiFOLD - Document Analysis and Correction')

        # Calculate window size (80% of screen width and height)
        window_width = int(screen_geometry.width() * 0.8)
        window_height = int(screen_geometry.height() * 0.8)

        # Ensure the window fits within the screen bounds
        window_width = min(window_width, screen_geometry.width())
        window_height = min(window_height, screen_geometry.height())

        # (width, height)
        self.setGeometry(100, 100, window_width, window_height)

        ################################# CREATE MENU BAR #################################
        self.create_menu_bar()

        ################################# STATUS BAR #################################
        self.statusBar = QStatusBar()
        self.setStatusBar(self.statusBar)
        self.statusBar.showMessage("Ready")

        ################################# CENTRAL WIDGET #################################
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QHBoxLayout(central_widget)

        ################################# DIRECTORIES FOR RECORDING #################################
        # Initialize directory attributes first
        self.recordings_dir = "recordings"
        self.transcriptions_dir = "transcriptions"
        
        # Create directories if they don't exist
        for directory in [self.recordings_dir, self.transcriptions_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
        
        # Initialize other attributes
        self.recording_thread = None
        self.last_transcription = ""
        self.current_audio_file = ""
        self.current_timestamp = ""
        
        ################################# IMAGE VIEWING VARIABLES #################################
        # Minimum zoom level (percentage)
        self.min_zoom = 10    
        # Maximum zoom level (percentage) 
        self.max_zoom = 300  
        # Current zoom level (percentage)   
        self.current_zoom = 100 
        # Zoom step percentage 
        self.zoom_step = 5
        
        # Initialize variables
        self.image_file = None  # Store the loaded image
        self.original_image = None  # Store the original image before Restormer
        self.restormer_applied = False  # Track if Restormer is applied
        self.start_time = None  # For timer
        self.end_time = None    # For timer

        ################################# LEFT PANEL #################################
        left_panel = QVBoxLayout()
        
        # Document Controls Group
        doc_controls_group = QGroupBox("Document Controls")
        doc_controls_layout = QVBoxLayout()
        
        # Document Processing Options
        processing_options_group = QGroupBox("Processing Options")
        processing_options_layout = QVBoxLayout()
        
        # Restormer toggle button
        restormer_layout = QHBoxLayout()
        restormer_label = QLabel("Image Restoration:")
        self.restormer_toggle = QCheckBox("Apply Restormer")
        self.restormer_toggle.setToolTip("Apply Restormer image restoration model to enhance document quality")
        self.restormer_toggle.stateChanged.connect(self.toggle_restormer)
        restormer_layout.addWidget(restormer_label)
        restormer_layout.addWidget(self.restormer_toggle)
        
        # Add to processing options
        processing_options_layout.addLayout(restormer_layout)
        processing_options_group.setLayout(processing_options_layout)
        
        # Document Viewer
        image_viewer_group = QGroupBox("Document Viewer")
        image_viewer_layout = QVBoxLayout()
        
        # Scroll area for the image
        self.scroll_area = QScrollArea()
        self.scroll_area.setWidgetResizable(True)
        self.scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        self.scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        
        # Create image label
        self.image = QLabel()
        self.image_height = 600  # Default height
        self.image_width = 800   # Default width
        self.image.setAlignment(Qt.AlignCenter)
        self.image.setSizePolicy(QSizePolicy.Ignored, QSizePolicy.Ignored)
        self.image.setScaledContents(True)
        
        # Add image label to scroll area
        self.scroll_area.setWidget(self.image)
        
        # Enable mouse wheel events for the scroll area
        self.scroll_area.setMouseTracking(True)
        self.scroll_area.wheelEvent = self.wheel_event
        
        # Zoom Controls
        zoom_controls_layout = QHBoxLayout()
        
        self.zoom_in_button = QPushButton()
        self.zoom_in_button.setIcon(self.style().standardIcon(QStyle.SP_ArrowUp))
        self.zoom_in_button.setToolTip('Zoom In (+)')
        self.zoom_in_button.clicked.connect(self.zoom_in)
        
        self.zoom_out_button = QPushButton()
        self.zoom_out_button.setIcon(self.style().standardIcon(QStyle.SP_ArrowDown))
        self.zoom_out_button.setToolTip('Zoom Out (-)')
        self.zoom_out_button.clicked.connect(self.zoom_out)
        
        self.reset_button = QPushButton("100%")
        self.reset_button.setToolTip('Reset Zoom')
        self.reset_button.clicked.connect(self.reset_zoom)
        
        self.zoom_label = QLabel(f'{self.current_zoom}%')
        
        zoom_controls_layout.addWidget(QLabel("Zoom:"))
        zoom_controls_layout.addWidget(self.zoom_out_button)
        zoom_controls_layout.addWidget(self.zoom_label)
        zoom_controls_layout.addWidget(self.zoom_in_button)
        zoom_controls_layout.addWidget(self.reset_button)
        
        # Add slider for zoom control
        self.zoom_slider = QSlider(Qt.Horizontal)
        self.zoom_slider.setMinimum(self.min_zoom)
        self.zoom_slider.setMaximum(self.max_zoom)
        self.zoom_slider.setValue(self.current_zoom)
        self.zoom_slider.setTickPosition(QSlider.TicksBelow)
        self.zoom_slider.setTickInterval(25)
        self.zoom_slider.valueChanged.connect(self.slider_zoom)
        
        # Add to image viewer layout
        image_viewer_layout.addWidget(self.scroll_area)
        image_viewer_layout.addLayout(zoom_controls_layout)
        image_viewer_layout.addWidget(self.zoom_slider)
        image_viewer_group.setLayout(image_viewer_layout)
        
        # Add groups to left panel
        left_panel.addWidget(processing_options_group)
        left_panel.addWidget(image_viewer_group)
        
        # Load a sample image
        self.original_pixmap = self.create_sample_image(self.image_width, self.image_height)
        self.image.setPixmap(self.original_pixmap)
        self.update_image()

        ################################# RIGHT PANEL #################################
        right_panel = QVBoxLayout()
        
        # Timer Group
        time_group = QGroupBox("Processing Time")
        time_layout = QVBoxLayout()
        
        # Timer controls
        timer_controls = QHBoxLayout()
        self.start_button = QPushButton('Start Time')
        self.start_button.setIcon(self.style().standardIcon(QStyle.SP_MediaPlay))
        self.start_button.clicked.connect(self.start_timer)
        
        self.end_button = QPushButton('End Time')
        self.end_button.setIcon(self.style().standardIcon(QStyle.SP_MediaStop))
        self.end_button.clicked.connect(self.end_timer)
        self.end_button.setEnabled(False)  # Disabled until start is clicked
        
        timer_controls.addWidget(self.start_button)
        timer_controls.addWidget(self.end_button)
        
        # Timer display
        timer_display = QGridLayout()
        timer_display.addWidget(QLabel("Start:"), 0, 0)
        self.start_label = QLabel('Not started')
        timer_display.addWidget(self.start_label, 0, 1)
        
        timer_display.addWidget(QLabel("End:"), 1, 0)
        self.end_label = QLabel('Not ended')
        timer_display.addWidget(self.end_label, 1, 1)
        
        timer_display.addWidget(QLabel("Elapsed:"), 2, 0)
        self.elapsed_time = QLabel('00:00:00.000')
        self.elapsed_time.setStyleSheet("font-weight: bold;")
        timer_display.addWidget(self.elapsed_time, 2, 1)
        
        time_layout.addLayout(timer_controls)
        time_layout.addLayout(timer_display)
        time_group.setLayout(time_layout)
        
        # OCR Output Group
        ocr_group = QGroupBox("OCR Output")
        ocr_layout = QVBoxLayout()
        
        # Text edit for OCR output
        self.ocr_text_edit = QTextEdit()
        self.ocr_text_edit.setReadOnly(False)
        self.ocr_text_edit.setPlaceholderText("OCR text will appear here after processing a document")
        
        # Add to OCR layout
        ocr_layout.addWidget(self.ocr_text_edit)
        ocr_group.setLayout(ocr_layout)
        
        # Speech Input Group
        speech_group = QGroupBox("Speech Input")
        speech_layout = QVBoxLayout()
        
        # Recording controls
        recording_controls = QHBoxLayout()
        
        self.record_button = QPushButton('Start Recording')
        self.record_button.setIcon(self.style().standardIcon(QStyle.SP_DialogApplyButton))
        self.record_button.clicked.connect(self.toggle_recording)
        
        self.insert_button = QPushButton('Insert at Cursor')
        self.insert_button.setIcon(self.style().standardIcon(QStyle.SP_ArrowRight))
        self.insert_button.clicked.connect(self.insert_at_cursor)
        self.insert_button.setEnabled(False)
        
        self.replace_button = QPushButton('Replace Selected')
        self.replace_button.setIcon(self.style().standardIcon(QStyle.SP_BrowserReload))
        self.replace_button.clicked.connect(self.replace_selected)
        self.replace_button.setEnabled(False)
        
        recording_controls.addWidget(self.record_button)
        recording_controls.addWidget(self.insert_button)
        recording_controls.addWidget(self.replace_button)
        
        # Save location display
        save_location = QHBoxLayout()
        self.directory_button = QPushButton('Change Location')
        self.directory_button.clicked.connect(self.change_directory)
        save_location.addWidget(self.directory_button)
        
        # Transcription display
        self.transcription_label = QLabel("Transcription will appear here")
        self.transcription_label.setWordWrap(True)
        self.transcription_label.setStyleSheet("background-color: #f0f0f0; padding: 5px; border: 1px solid #ddd;")
        
        # Add to speech layout
        speech_layout.addLayout(recording_controls)
        speech_layout.addLayout(save_location)
        speech_layout.addWidget(self.transcription_label)
        speech_group.setLayout(speech_layout)
        
        # Add groups to right panel
        right_panel.addWidget(time_group)
        right_panel.addWidget(ocr_group, 1)  # Give OCR output more space
        right_panel.addWidget(speech_group)
        
        ################################# ADD PANELS TO MAIN LAYOUT #################################
        main_layout.addLayout(left_panel, 1)
        main_layout.addLayout(right_panel, 1)
        
        # Show the main window
        self.show()

    def create_menu_bar(self):
        """Create menu bar with file and help options"""
        menu_bar = self.menuBar()
        
        # File menu
        file_menu = menu_bar.addMenu("File")
        
        # Open action
        open_action = QAction("Open Document", self)
        open_action.setShortcut("Ctrl+O")
        open_action.triggered.connect(self.load_document)
        file_menu.addAction(open_action)
        
        # Save action
        save_action = QAction("Save OCR Text", self)
        save_action.setShortcut("Ctrl+S")
        save_action.triggered.connect(self.save)
        file_menu.addAction(save_action)
        
        file_menu.addSeparator()
        
        # Exit action
        exit_action = QAction("Exit", self)
        exit_action.setShortcut("Ctrl+Q")
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)
        
        # Tools menu
        tools_menu = menu_bar.addMenu("Tools")
        
        # Recording action
        record_action = QAction("Start Recording", self)
        record_action.triggered.connect(self.toggle_recording)
        tools_menu.addAction(record_action)
        
        # Timer action
        timer_action = QAction("Start Timer", self)
        timer_action.triggered.connect(self.start_timer)
        tools_menu.addAction(timer_action)
        
        # Help menu
        help_menu = menu_bar.addMenu("Help")
        
        # About action
        about_action = QAction("About MultiFOLD", self)
        about_action.triggered.connect(self.show_about)
        help_menu.addAction(about_action)

    def show_about(self):
        """Show information about the application"""
        QMessageBox.about(self, "About MultiFOLD", 
                         "MultiFOLD: Document Analysis and Correction Tool\n\n"
                         "This application helps with document processing, OCR, and speech-to-text conversion.\n\n"
                         "Use it to analyze documents, correct OCR errors, and track processing time.")

    def toggle_restormer(self, state):
        """Toggle Restormer image restoration on/off"""
        if self.image_file is None:
            if state == Qt.Checked:
                self.statusBar.showMessage("No image loaded. Load an image first to apply Restormer.")
                self.restormer_toggle.setChecked(False)
            return
        
        if state == Qt.Checked:
            self.statusBar.showMessage("Applying Restormer image restoration...")
            # Store the current original image if not already stored
            if not self.restormer_applied:
                self.original_image = self.image_file.copy() if hasattr(self.image_file, 'copy') else self.image_file
            
            # Apply Restormer (placeholder implementation)
            self.apply_restormer()
            self.restormer_applied = True
        else:
            # Restore original image
            if self.original_image is not None:
                self.statusBar.showMessage("Restoring original image...")
                self.image_file = self.original_image
                self.display_image(self.image_file)
                self.restormer_applied = False

    def apply_restormer(self):
        """Apply Restormer image restoration model to the current image"""
        if self.image_file is None:
            return
            
        try:
            # Convert to numpy array if it's a PIL Image
            if isinstance(self.image_file, Image.Image):
                img_np = np.array(self.image_file)
            else:
                img_np = self.image_file
                
            # Ensure the image is in RGB format
            if len(img_np.shape) == 2:  # If grayscale, convert to RGB
                img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2RGB)
            elif len(img_np.shape) == 3 and img_np.shape[2] == 4:  # If RGBA, convert to RGB
                img_np = cv2.cvtColor(img_np, cv2.COLOR_RGBA2RGB)
            
            # Define model parameters (based on the shared code)
            task = 'Motion_Deblurring'  # You can make this configurable
            parameters = {
                'inp_channels': 3, 
                'out_channels': 3, 
                'dim': 48, 
                'num_blocks': [4, 6, 6, 8], 
                'num_refinement_blocks': 4, 
                'heads': [1, 2, 4, 8], 
                'ffn_expansion_factor': 2.66, 
                'bias': False, 
                'LayerNorm_type': 'WithBias', 
                'dual_pixel_task': False
            }
            
            # Adjust parameters based on the task
            if task == 'Real_Denoising':
                parameters['LayerNorm_type'] = 'BiasFree'
                weights_path = 'pretrained_models/real_denoising.pth'
            elif task == 'Single_Image_Defocus_Deblurring':
                weights_path = 'pretrained_models/single_image_defocus_deblurring.pth'
            elif task == 'Motion_Deblurring':
                weights_path = 'pretrained_models/motion_deblurring.pth'
            elif task == 'Deraining':
                weights_path = 'pretrained_models/deraining.pth'
            
            # Load the model
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = Restormer(**parameters)
            model.to(device)
            
            # Load weights
            checkpoint = torch.load(weights_path, map_location=device)
            model.load_state_dict(checkpoint['params'])
            model.eval()
            
            # Prepare input tensor
            img_multiple_of = 8
            input_ = torch.from_numpy(img_np).float().div(255.).permute(2, 0, 1).unsqueeze(0).to(device)
            
            # Pad the input if not multiple of 8
            h, w = input_.shape[2], input_.shape[3]
            H = ((h + img_multiple_of) // img_multiple_of) * img_multiple_of
            W = ((w + img_multiple_of) // img_multiple_of) * img_multiple_of
            padh = H - h if h % img_multiple_of != 0 else 0
            padw = W - w if w % img_multiple_of != 0 else 0
            input_ = F.pad(input_, (0, padw, 0, padh), 'reflect')
            
            # Process with model
            with torch.no_grad():
                restored = model(input_)
                restored = torch.clamp(restored, 0, 1)
                
                # Unpad the output
                restored = restored[:, :, :h, :w]
                
                # Convert to numpy
                restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
                restored = img_as_ubyte(restored[0])
            
            # Convert back to PIL Image and update display
            restored_pil = Image.fromarray(restored)
            self.image_file = restored_pil
            
            # Display the restored image
            self.display_image(restored_pil)
            
            self.statusBar.showMessage(f"Restormer {task} applied successfully")
            
        except ImportError as e:
            self.statusBar.showMessage(f"Error: Missing dependencies - {str(e)}")
            QMessageBox.warning(self, "Restormer Error", 
                            f"Missing dependencies: {str(e)}\nPlease install required packages.")
            self.restormer_toggle.setChecked(False)
        except FileNotFoundError as e:
            self.statusBar.showMessage("Error: Restormer model weights not found")
            QMessageBox.warning(self, "Restormer Error", 
                            f"Model weights not found: {str(e)}\nPlease download the model files.")
            self.restormer_toggle.setChecked(False)
        except Exception as e:
            self.statusBar.showMessage(f"Error applying Restormer: {str(e)}")
            QMessageBox.warning(self, "Restormer Error", f"Could not apply Restormer: {str(e)}")
            self.restormer_toggle.setChecked(False)

    # def apply_restormer(self):
    #     """Apply Restormer image restoration model to the current image"""
    #     # This is a placeholder implementation since we don't have actual Restormer integration
    #     # In a real implementation, this would call the Restormer model
        
    #     if self.image_file is None:
    #         return
        
    #     try:
    #         # Convert to numpy array if it's a PIL Image
    #         if isinstance(self.image_file, Image.Image):
    #             img_np = np.array(self.image_file)
    #         else:
    #             img_np = self.image_file
                
    #         # Placeholder Restormer effect - just apply simple image enhancement
    #         # In reality, you would call the actual Restormer model here
            
    #         # Convert to grayscale if it's color
    #         if len(img_np.shape) == 3 and img_np.shape[2] >= 3:
    #             gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    #         else:
    #             gray = img_np
                
    #         # Apply simple enhancement (adaptive histogram equalization)
    #         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    #         enhanced = clahe.apply(gray)
            
    #         # Convert back to RGB for display
    #         enhanced_rgb = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2RGB)
            
    #         # Convert back to PIL Image
    #         enhanced_pil = Image.fromarray(enhanced_rgb)
    #         self.image_file = enhanced_pil
            
    #         # Display the enhanced image
    #         self.display_image(enhanced_pil)
            
    #         self.statusBar.showMessage("Restormer applied (simulated enhancement)")
            
    #     except Exception as e:
    #         self.statusBar.showMessage(f"Error applying Restormer: {str(e)}")
    #         QMessageBox.warning(self, "Restormer Error", f"Could not apply Restormer: {str(e)}")
    #         self.restormer_toggle.setChecked(False)

    ############### Document Processing Methods ###############
    def remove_extra_newlines(self, text):
        cleaned_text = re.sub(r'\n+', '\n', text).strip()
        return cleaned_text

    def load_document(self):
        # Open file dialog to load an image or document
        options = QFileDialog.Options()
        file, _ = QFileDialog.getOpenFileName(self, "Open Document", "", 
                                             "All Files (*);;Image Files (*.png; *.jpg; *.jpeg);;Word Documents (*.docx);;PDF Files (*.pdf)", 
                                             options=options)

        if file:
            self.statusBar.showMessage(f"Processing document: {os.path.basename(file)}")
            # Reset Restormer toggle when loading a new document
            self.restormer_toggle.setChecked(False)
            self.restormer_applied = False
            self.original_image = None
            
            self.process_document(file)
            self.statusBar.showMessage(f"Document processed: {os.path.basename(file)}")

    def process_document(self, file):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Process as image file
            self.image_file = Image.open(file)
            self.display_image(self.image_file)
            ocr_text = pytesseract.image_to_string(self.image_file)
            
            ocr_text = self.remove_extra_newlines(ocr_text)
            self.ocr_text_edit.setText(ocr_text)
            
        elif file.lower().endswith('.docx'):
            # Process as Word document
            self.process_word_document(file)
            
        elif file.lower().endswith('.pdf'):
            # Process as PDF document
            self.process_pdf_document(file)

    def process_word_document(self, file):
        doc = Document(file)
        full_text = ''
        for para in doc.paragraphs:
            full_text += para.text + '\n'

        full_text = self.remove_extra_newlines(full_text)
        self.ocr_text_edit.setText(full_text)
        
        # Create a placeholder image for the document
        self.create_document_preview("Word Document")

    def process_pdf_document(self, file):
        # Convert PDF pages to images
        pages = convert_from_path(file, 600)
        if pages:
            # Use the first page for OCR and display
            self.image_file = pages[0]
            self.display_image(self.image_file)

            # Perform OCR on the first page image
            ocr_text = pytesseract.image_to_string(self.image_file)
            
            if ocr_text.strip() == "":
                ocr_text = "No text detected in this document."

            ocr_text = self.remove_extra_newlines(ocr_text)
            self.ocr_text_edit.setText(ocr_text)

    def save(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save OCR Text", "", 
                                                 "Text Files (*.txt);;All Files (*)", 
                                                 options=options)

        if file_name:
            try:
                text = self.ocr_text_edit.toPlainText()
                
                # Ensure file has .txt extension
                if not file_name.lower().endswith('.txt'):
                    file_name += '.txt'
                    
                with open(file_name, 'w') as f:
                    f.write(text)
                    
                self.statusBar.showMessage(f"Text saved to {os.path.basename(file_name)}")
            except Exception as e:
                self.statusBar.showMessage(f"Error saving file: {str(e)}")
                QMessageBox.critical(self, "Save Error", f"Could not save file: {str(e)}")

    ############### Image Display Methods ###############
    def display_image(self, image):
        """Display image in the QLabel"""
        # Convert PIL Image to QPixmap
        self.original_pixmap = self.pil_image_to_pixmap(image)
        self.image.setPixmap(self.original_pixmap)
        self.update_image()

    def create_document_preview(self, doc_type):
        """Create a placeholder image for non-image documents"""
        # Create a blank QImage
        qimage = QImage(800, 600, QImage.Format_RGB32)
        qimage.fill(Qt.white)
        
        # Create a QPainter to draw on the image
        painter = QPainter(qimage)
        painter.setPen(Qt.black)
        painter.setFont(QFont("Arial", 20))
        
        # Draw text in the center
        painter.drawText(qimage.rect(), Qt.AlignCenter, f"{doc_type}\nProcessed Successfully")
        
        # Draw a border
        painter.drawRect(10, 10, qimage.width() - 20, qimage.height() - 20)
        painter.end()
        
        # Convert to QPixmap and display
        self.original_pixmap = QPixmap.fromImage(qimage)
        self.image.setPixmap(self.original_pixmap)
        self.update_image()

    def pil_image_to_pixmap(self, pil_image):
        """Convert a PIL Image to QPixmap"""
        # Convert PIL Image to RGB if it's not already
        if pil_image.mode != "RGB":
            pil_image = pil_image.convert("RGB")
            
        image_data = pil_image.tobytes("raw", "RGB")
        q_image = QImage(image_data, pil_image.width, pil_image.height, pil_image.width * 3, QImage.Format_RGB888)
        return QPixmap.fromImage(q_image)

    def create_sample_image(self, width, height):
        """Create a sample image if no image is loaded"""
        # Create a blank image with a grid pattern
        image = QImage(width, height, QImage.Format_RGB32)
        image.fill(Qt.white)
        
        # Create a QPainter to draw on the image
        painter = QPainter(image)
        painter.setPen(Qt.lightGray)
        
        # Draw a grid
        for x in range(0, width, 50):
            painter.drawLine(x, 0, x, height)
        
        for y in range(0, height, 50):
            painter.drawLine(0, y, width, y)
        
        # Draw text in the center
        painter.setPen(Qt.black)
        painter.setFont(QFont("Arial", 18))
        painter.drawText(image.rect(), Qt.AlignCenter, "MultiFOLD\nLoad a document to begin")
        
        painter.end()
        
        # Create a pixmap from the image
        return QPixmap.fromImage(image)

    ############### Zoom Control Methods ###############
    def update_image(self):
        """Update the image with the current zoom level"""
        # Calculate new dimensions
        new_width = int((self.original_pixmap.width() * self.current_zoom) / 100)
        new_height = int((self.original_pixmap.height() * self.current_zoom) / 100)
        
        # Set fixed size for the label based on zoomed dimensions
        self.image.setFixedSize(new_width, new_height)
        
        # Set the pixmap
        self.image.setPixmap(self.original_pixmap)
        
        # Update zoom label
        self.zoom_label.setText(f'{self.current_zoom}%')
        
        # Update slider without triggering valueChanged signal
        self.zoom_slider.blockSignals(True)
        self.zoom_slider.setValue(self.current_zoom)
        self.zoom_slider.blockSignals(False)
    
    def zoom_in(self):
        """Increase zoom level"""
        if self.current_zoom < self.max_zoom:
            self.current_zoom += self.zoom_step
            self.update_image()
    
    def zoom_out(self):
        """Decrease zoom level, but not below minimum"""
        if self.current_zoom > self.min_zoom:
            self.current_zoom -= self.zoom_step
            self.update_image()
    
    def reset_zoom(self):
        """Reset zoom to 100%"""
        self.current_zoom = 100
        self.update_image()
    
    def slider_zoom(self, value):
        """Handle zoom from slider"""
        self.current_zoom = value
        self.update_image()
    
    def wheel_event(self, event):
        """Handle mouse wheel events for zooming"""
        if event.modifiers() & Qt.ControlModifier:
            # Zoom with Ctrl+Wheel
            delta = event.angleDelta().y()
            if delta > 0:
                self.zoom_in()
            else:
                self.zoom_out()
            event.accept()
        else:
            # Default scroll behavior
            QScrollArea.wheelEvent(self.scroll_area, event)
    
    def keyPressEvent(self, event):
        """Handle keyboard shortcuts for zooming"""
        if event.key() == Qt.Key_Plus or event.key() == Qt.Key_Equal:
            self.zoom_in()
        elif event.key() == Qt.Key_Minus:
            self.zoom_out()
        elif event.key() == Qt.Key_0:
            self.reset_zoom()
        else:
            super().keyPressEvent(event)

    ############### Timer Methods ###############
    def start_timer(self):
        self.start_time = QDateTime.currentDateTime()
        self.start_label.setText(f'{self.start_time.toString("hh:mm:ss.zzz")}')
        self.start_button.setEnabled(False)
        self.end_button.setEnabled(True)
        self.elapsed_time.setText('00:00:00.000')
        self.statusBar.showMessage("Timer started")
    
    def end_timer(self):
        if self.start_time:
            self.end_time = QDateTime.currentDateTime()
            self.end_label.setText(f'{self.end_time.toString("hh:mm:ss.zzz")}')
            self.calculate_difference()
            self.start_button.setEnabled(True)
            self.end_button.setEnabled(False)
            self.statusBar.showMessage("Timer stopped")
    
    def calculate_difference(self):
        if self.start_time and self.end_time:
            # Calculate time difference in milliseconds
            milliseconds = self.start_time.msecsTo(self.end_time)
            
            # Format the time difference in hh:mm:ss.msms format
            hours = milliseconds // 3600000
            minutes = (milliseconds % 3600000) // 60000
            seconds = (milliseconds % 60000) // 1000
            ms = milliseconds % 1000
            
            formatted_time = f"{hours:02d}:{minutes:02d}:{seconds:02d}.{ms:03d}"
            # Display result in the label
            self.elapsed_time.setText(f'{formatted_time}')

    ############### Speech Recording Methods ###############
    def get_timestamped_filename(self):
        """Generate a filename with current timestamp"""
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        audio_file = os.path.join(self.recordings_dir, f"audio_{timestamp}.wav")
        return audio_file, timestamp
    
    def toggle_recording(self):
        if self.recording_thread is None:
            # Start recording with timestamped filename
            audio_file, self.current_timestamp = self.get_timestamped_filename()
            self.current_audio_file = audio_file
            
            self.statusBar.showMessage(f"Recording... (File: audio_{self.current_timestamp}.wav)")
            self.record_button.setText("Stop Recording")
            self.record_button.setIcon(self.style().standardIcon(QStyle.SP_MediaStop))
            self.recording_thread = RecordingThread(audio_file)
            self.recording_thread.finished.connect(self.on_transcription_finished)
            self.recording_thread.start()
        else:
            # Stop recording
            self.statusBar.showMessage("Processing speech...")
            self.record_button.setText("Start Recording")
            self.record_button.setIcon(self.style().standardIcon(QStyle.SP_DialogApplyButton))
            self.recording_thread.stop()
            self.recording_thread = None
    
    def on_transcription_finished(self, text, audio_file):
        """Handle completed transcription and save to file"""
        self.statusBar.showMessage(f"Transcription complete: {os.path.basename(audio_file)}")
        self.last_transcription = text
        self.transcription_label.setText(text)
        self.insert_button.setEnabled(True)
        self.replace_button.setEnabled(True)
        
        # Save transcription to a JSON file with same timestamp
        timestamp = os.path.basename(audio_file).replace("audio_", "").replace(".wav", "")
        transcription_file = os.path.join(self.transcriptions_dir, f"transcription_{timestamp}.json")
        
        transcription_data = {
            "timestamp": timestamp,
            "audio_file": audio_file,
            "transcription": text,
            "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        with open(transcription_file, 'w') as f:
            json.dump(transcription_data, f, indent=4)
            
        self.statusBar.showMessage(f"Saved: {os.path.basename(audio_file)} and {os.path.basename(transcription_file)}")
    
    def insert_at_cursor(self):
        if self.last_transcription:
            cursor = self.ocr_text_edit.textCursor()
            cursor.insertText(self.last_transcription)
            self.statusBar.showMessage("Text inserted at cursor position")
    
    def replace_selected(self):
        if self.last_transcription:
            cursor = self.ocr_text_edit.textCursor()
            if cursor.hasSelection():
                cursor.insertText(self.last_transcription)
                self.statusBar.showMessage("Selected text replaced")
            else:
                self.statusBar.showMessage("No text selected to replace")
    
    def change_directory(self):
        """Change the directory where recordings and transcriptions are saved"""
        parent_dir = QFileDialog.getExistingDirectory(self, "Select Directory")
        if parent_dir:
            self.recordings_dir = os.path.join(parent_dir, "recordings")
            self.transcriptions_dir = os.path.join(parent_dir, "transcriptions")
            
            # Create directories if they don't exist
            for directory in [self.recordings_dir, self.transcriptions_dir]:
                if not os.path.exists(directory):
                    os.makedirs(directory)
                    
            self.statusBar.showMessage(f"Changed save location to {os.path.abspath(self.recordings_dir)}")
    
    def closeEvent(self, event):
        # Ask user for confirmation before closing
        reply = QMessageBox.question(self, 'Exit Confirmation',
                                    'Are you sure you want to exit?',
                                    QMessageBox.Yes | QMessageBox.No,
                                    QMessageBox.No)

        if reply == QMessageBox.Yes:
            # If recording is active, stop it
            if self.recording_thread is not None:
                self.recording_thread.stop()
                self.recording_thread = None
            event.accept()
        else:
            event.ignore()


if __name__ == "__main__":
    # Make sure the application looks good on high DPI displays
    QApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True)
    QApplication.setAttribute(Qt.AA_UseHighDpiPixmaps, True)
    
    app = QApplication(sys.argv)
    
    # Set application style for a more modern look
    app.setStyle("Fusion")
    
    # Apply a palette to get a consistent look across platforms
    palette = QPalette()
    palette.setColor(QPalette.Window, QColor(240, 240, 240))
    palette.setColor(QPalette.WindowText, QColor(0, 0, 0))
    palette.setColor(QPalette.Base, QColor(255, 255, 255))
    palette.setColor(QPalette.AlternateBase, QColor(230, 230, 230))
    palette.setColor(QPalette.ToolTipBase, QColor(255, 255, 220))
    palette.setColor(QPalette.ToolTipText, QColor(0, 0, 0))
    palette.setColor(QPalette.Text, QColor(0, 0, 0))
    palette.setColor(QPalette.Button, QColor(240, 240, 240))
    palette.setColor(QPalette.ButtonText, QColor(0, 0, 0))
    palette.setColor(QPalette.BrightText, QColor(255, 0, 0))
    palette.setColor(QPalette.Highlight, QColor(42, 130, 218))
    palette.setColor(QPalette.HighlightedText, QColor(255, 255, 255))
    app.setPalette(palette)
    
    window = MultiFOLD()
    window.show()
    sys.exit(app.exec_())